In [ ]:
import Pkg
Pkg.activate(".")

# not for the faint of heart!
# Pkg.update()

pkgs = [
"ArgParse",
"Base64",
"BioSequences",
"DataFrames",
"Dates",
"DelimitedFiles",
"FASTX",
"Statistics",
"StatsPlots",
"uCSV",
"Revise",
"StatsBase"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), SRR_paths)
# filter out samples that have already been processed
# SRR_paths = filter(x -> !isfile(joinpath(x, basename(x) * ".contig_info.tsv")), SRR_paths)

In [ ]:
# mamba install -c conda-forge -c bioconda virsorter=2
# mamba create -n vs2 -c conda-forge -c bioconda virsorter=2
# mamba activate vs2

In [ ]:
virsorter_db_dir = mkpath("$(homedir())/workspace/virsorter2")

# run(`virsorter setup -d $(virsorter_db_dir) -j 4`)
# run(`https://osf.io/v46sc/download`)

db_tgz_path = joinpath(virsorter_db_dir, "db.tgz")
if !isfile(db_tgz_path)
    download("https://osf.io/v46sc/download", joinpath(virsorter_db_dir, "db.tgz"))
end
db_dir = replace(db_tgz_path, ".tgz" => "")
if !isdir(db_dir)
    run(`tar -xzf $(db_tgz_path) --directory=$(virsorter_db_dir)`)
end

In [ ]:
# mamba create -n vs2 -c conda-forge -c bioconda virsorter=2
# $  mamba activate vs2
# (vs2) 2023-06-06T16:26:44 jovyan@76eefed4ebe2:~
# $ virsorter config --init-source --db-dir=/home/jovyan/workspace/virsorter2/db
# [2023-06-06 16:27 INFO] VirSorter 2.2.4
# [2023-06-06 16:27 INFO] /opt/conda/envs/vs2/bin/virsorter config --init-source --db-dir=/home/jovyan/workspace/virsorter2/db
# [2023-06-06 16:27 INFO] saving /home/jovyan/workspace/virsorter2/db as DBDIR to config file /opt/conda/envs/vs2/lib/python3.10/site-packages/virsorter/template-config.yaml
# # run(`virsorter config --init-source --db-dir=$(db_dir)`)

In [ ]:
virsorter_db_dir_path = joinpath(virsorter_db_dir, "db")

In [ ]:
for SRR_path in SRR_paths
    SRR = basename(SRR_path)
    assembled_fasta = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna")
    virsorter_dir = mkpath(joinpath(SRR_path, "virsorter"))
    outfile = joinpath(virsorter_dir, "final-viral-combined.fa")
    if !isfile(outfile)
        # run(`conda run --no-capture-output -n vs2 virsorter run --db-dir $(virsorter_db_dir_path) -w $(virsorter_dir) -i $(assembled_fasta) --min-length 1 -j $(Sys.CPU_THREADS-1) all`)
        run(`podman-hpc run quay.io/biocontainers/virsorter:2.2.4--pyhdfd78af_0 virsorter run --db-dir $(virsorter_db_dir_path) -w $(virsorter_dir) -i $(assembled_fasta) --min-length 1 -j $(Sys.CPU_THREADS-1) all`)
    else
        println("$outfile already exists, skipping...")
    end
end

In [ ]:
# 30 minutes per run

# works in standalone conda environment but not general installation
# run(`virsorter run -w $(virsorter_dir) -i $(assembled_fasta) --min-length 1 -j $(Sys.CPU_THREADS) all`)

# virsorter_dir = mkpath(joinpath(SRR_path, "virsorter"))

# outfile = joinpath(virsorter_dir, "final-viral-combined.fa")
# if !isfile(outfile)
#     run(`conda run --no-capture-output -n vs2 virsorter run -w $(virsorter_dir) -i $(assembled_fasta) --min-length 1 -j $(Sys.CPU_THREADS-1) all`)
# else
#     println("$outfile already exists, skipping...")
# end
    
# [2023-06-06 16:59 INFO] Step 2 - extract-feature finished.
# [2023-06-06 17:00 INFO] 
#             ====> VirSorter run (provirus mode) finished.
#             # of full    seqs (>=2 genes) as viral:     30
#             # of partial seqs (>=2 genes) as viral:     0
#             # of short   seqs (< 2 genes) as viral:     0

#             Useful output files:
#                 final-viral-score.tsv       ==> score table
#                 final-viral-combined.fa     ==> all viral seqs
#                 final-viral-boundary.tsv    ==> table with boundary info
                
            
#                 Suffix is added to seq names in final-viral-combined.fa:
#                 full    seqs (>=2 genes) as viral:      ||full
#                 partial seqs (>=2 genes) as viral:      ||partial
#                 short   seqs (< 2 genes) as viral:      ||lt2gene
                
                
#             NOTES:
#             Users can further screen the results based on the following 
#                 columns in final-viral-score.tsv:
#                 - contig length (length) 
#                 - hallmark gene count (hallmark)
#                 - viral gene % (viral) 
#                 - cellular gene % (cellular)
#             The group field in final-viral-score.tsv should NOT be used
#                 as reliable taxonomy info
#             We recommend this SOP/tutorial for quality control 
#                 (make sure to use the lastest version):
#                 https://dx.doi.org/10.17504/protocols.io.bwm5pc86

#             <====
            
# [2023-06-06 17:00 INFO] Step 3 - classify finished.